In [ ]:
if __name__ == '__main__':
    hmm_file = '# hmm file path'
    feat_scp = '# feature list file path'
    lexicon_file = '# lexicon file path'
    phones_file = '# phone list file path'
    insert_sil = True # whether to insert silenece at the beginning and the end
    
    # open phones_file and make up a list of phones
    phone_list = []
    with open(phones_file, mode = 'r') as f:
        for line in f:
            phone = line.split()[0]
            phone_list.append(phone)
            
    # open lexicon_file and make up a lexicon list with word, phones that comprise the word, the matching index of the phone sequence
    lexicon = []
    with open(lexicon_file, mode = 'r') as f:
        for line in f:
            word = line.split()[0]
            phones = line.split()[1:]
            if insert_sil:
                phones.insert(0, phone_list[0])
                phones.append(phone_list[0])
            # create the matching index sequences
            ph_int = []
            for ph in phones:
                if ph in phone_list:
                    ph_int.append(phone_list.index(ph))
                else:
                    sys.stderr.write('invalud phone %s' % (ph))
            lexicon.append({'word': word, 
                           'phones': phones, 
                           'ph_int': ph_int})
    
    # read HMM file 
    hmm = MonoPhoneHMM()
    hmm = hmm.load(hmm_file)
    
    # read feature file 
    with open(feat_scp, mode = 'r') as f:
        for line in f:
            parts = line.split()
            utt_id, feat_path = parts[0], parts[1]
            
            feat = np.fromfile(feat_path, dtype = np.float32)
            feat = feat.reshape(-1, hmm.num_dims)
            
            # execute word recognisation
            (result, detail)  = hmm.recognize(feat, lexicon)
            
            sys.stdout.write('%s %s\n' % (utt, ff))
            sys.stdout.write('Result = %s\n' % (result))
            sys.stdout.write('Ranking]\n')
            for res in detail:
                sys.stdout.write(' %s %f\n' % (res['word'], res['score']))
        
            